<a href="https://colab.research.google.com/github/anhduong77/Credit-Scoring-ML-project/blob/main/notebooks/3_split_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/project/data/model_data.csv',index_col='issue_d', low_memory=False)

In [ ]:
data_description = pd.read_csv('/content/drive/MyDrive/project/data/data_dictionary.csv')

In [ ]:
# check for missing values
df_null = df.isnull().sum().to_frame()
df_null['missing_percentage'] = (df_null[0] / df.shape[0]) * 100
df_null.sort_values(by='missing_percentage', ascending=False, inplace=True)
display(df_null.head(10))
print("--------------------------------------")
print("Total features with missing values:", df_null[df_null[0] > 0].shape[0])
print("Total features with no missing values:", df_null[df_null[0] == 0].shape[0])

,0,missing_percentage
member_id,1765426,100.000000
orig_projected_additional_accrued_interest,1757096,99.528159
hardship_payoff_balance_amount,1754877,99.402467
hardship_last_payment_amount,1754877,99.402467
payment_plan_start_date,1754877,99.402467
hardship_type,1754877,99.402467
hardship_status,1754877,99.402467
hardship_start_date,1754877,99.402467
deferral_term,1754877,99.402467
hardship_amount,1754877,99.402467


--------------------------------------
Total features with missing values: 113
Total features with no missing values: 37


In [ ]:
# drop features with more than 30% missing values
df = df.drop(columns=df_null[df_null['missing_percentage'] > 30].index)
df.shape

(1765426, 92)

In [ ]:
tar_col = 'loan_status'
num_col = df.select_dtypes(include='number').columns.tolist()
cat_col = df.select_dtypes(exclude='number').columns.tolist()
recoveries = df['recoveries'].copy()
cat_col.remove(tar_col)

# Tarcol

In [ ]:
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000

df.loc[df['recoveries'] != 0, tar_col].unique()

array(['Does not meet the credit policy. Status:Charged Off',
       'Charged Off'], dtype=object)

In [ ]:
# group the target variable
df[tar_col].value_counts().to_frame().reset_index()

,loan_status,count
0,Fully Paid,1029307
1,Current,451136
2,Charged Off,259692
3,Late (31-120 days),14246
4,In Grace Period,5517
5,Late (16-30 days),2746
6,Does not meet the credit policy. Status:Fully Paid,1988
7,Does not meet the credit policy. Status:Charged Off,761
8,Default,33


In [ ]:
label = df[tar_col].value_counts().index.values
bad_loan = label[[2, 7]]
df[tar_col] = df[tar_col].apply(lambda x: 1 if x in bad_loan else 0)

In [ ]:
df[df[tar_col] == 'Late (31-120 days)']

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
issue_d,,,,,,,,,,,,,,,,,,,,,


# Numerical columns

- There are five phases:
  + application: borrower submits loan request
  + funding: investor review and pledge funds
  + approval: platform approves the loan terms
  + finalization/labeling: loan is issued and repayment begins
=> so we will remove all the features in and after finalization and labeling phase to avoid data leakage

In [ ]:
# application = 1, funding = 2, approval = 3, finalization/labeling = 4
num_phase ={'loan_amnt': 1, 'funded_amnt': 2, 'funded_amnt_inv': 2, 'int_rate': 3, 'installment': 4,
            'anuual_inc': 1, 'dti': 1, 'delinq_2yrs': 1, ' 	fico_range_low': 1, 'fico_range_high': 1,
            'inq_last_6mths': 1, 'open_acc': 1, 'pub_rec': 1, 'revol_bal': 1, 'revol_util': 1,
            'total_acc': 1, 'out_prncp': 4, 'out_prncp_inv': 4, 'total_pymnt': 4, 'total_pymnt_inv': 4,
            'total_rec_prncp': 4, 'total_rec_int': 4, 'total_rec_late_fee': 4, 'recoveries': 4, 'collection_recovery_fee': 4,
            'last_pymnt_amnt': 4, 'last_fico_range_high': 4, 'last_fico_range_low': 4, 'collections_12_mths_ex_med': 1,
            'policy_code': 1, 'acc_now_delinq': 1, 'tot_coll_amt': 1, 'tot_cur_bal': 1, 'total_rev_hi_lim': 1, 'acc_open_past_24mths': 1,
            'avg_cur_bal': 1, 'bc_open_to_buy': 1, 'bc_util': 1, 'chargeoff_within_12_mths': 1, 'delinq_amnt': 1,
            'mo_sin_old_il_acct': 1, 'mo_sin_old_rev_tl_op': 1, 'mo_sin_rcnt_rev_tl_op': 1, 'mo_sin_rcnt_tl': 1, 'mort_acc': 1,
            'mths_since_recent_bc': 3, 'mths_since_recent_inq': 3, 'num_accts_ever_120_pd': 3, 'num_actv_bc_tl': 3, 'num_actv_rev_tl': 3,
            'num_bc_sats': 3, 'num_bc_tl': 3, 'num_il_tl': 3, 'num_op_rev_tl': 3, 'num_rev_accts': 3,
            'num_rev_tl_bal_gt_0': 1, 'num_sats': 1, 'num_tl_120dpd_2m': 1, 'num_tl_30dpd': 1, 'num_tl_90g_dpd_24m': 1,
            'num_tl_90g_dpd_24m': 1, 'pct_tl_nvr_dlq': 1, 'percent_bc_gt_75': 1, 'pub_rec_bankruptcies': 1, 'tax_liens': 1,
            'tot_hi_cred_lim': 1, 'total_bal_ex_mort': 1, 'total_bc_limit': 1, 'total_il_high_credit_limit': 1}

In [ ]:
for column in df.columns:
  if (column in num_phase) and (num_phase[column.strip()]) == 4:
    df = df.drop(column, axis=1)
    num_col.remove(column)
# remove unrelated features
unrelated_features = ['id', 'policy_code']
df = df.drop(columns=unrelated_features, axis=1)
num_col = [x for x in num_col if x not in unrelated_features]

# Categorical features

In [ ]:
df[cat_col].head(1)

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,pymnt_plan,url,purpose,...,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,disbursement_method,debt_settlement_flag
issue_d,,,,,,,,,,,,,,,,,,,,,
2007-06,36 months,A,A5,NaN,< 1 year,NONE,Not Verified,n,https://lendingclub.com/browse/loanDetail.action?loan_id=71623,other,...,100xx,NY,NaN,f,Jun-2010,Aug-2007,Individual,N,Cash,N


In [ ]:
cat_dictionary = []
for col in cat_col:
    cat_dictionary.append({
        'Name': col,
        'Cardinality': df[col].nunique(),
        'Description': data_description.loc[data_description['Column'] == col, 'Description'],
        'Values': df[col].unique()
    })
cat_dictionary = pd.DataFrame(cat_dictionary).sort_values(by='Cardinality', ascending=False)

In [ ]:
cat_dictionary

,Name,Cardinality,Description,Values
8,url,1765426,"18 URL for the LC page with listing data. Name: Description, dtype: object","[https://lendingclub.com/browse/loanDetail.action?loan_id=71623, https://lendingclub.com/browse/loanDetail.action?loan_id=88637, https://lendingclub.com/browse/loanDetail.action?loan_id=88046, https://lendingclub.com/browse/loanDetail.action?loan_id=85961, https://lendingclub.com/browse/loanDetail.action?loan_id=85818, https://lendingclub.com/browse/loanDetail.action?loan_id=85781, https://lendingclub.com/browse/loanDetail.action?loan_id=85675, https://lendingclub.com/browse/loanDetail.action?loan_id=84670, https://lendingclub.com/browse/loanDetail.action?loan_id=84098, https://lendingclub.com/browse/loanDetail.action?loan_id=83979, https://lendingclub.com/browse/loanDetail.action?loan_id=83489, https://lendingclub.com/browse/loanDetail.action?loan_id=83185, https://lendingclub.com/browse/loanDetail.action?loan_id=74014, https://lendingclub.com/browse/loanDetail.action?loan_id=76629, https://lendingclub.com/browse/loanDetail.action?loan_id=77792, https://lendingclub.com/browse/loan..."
3,emp_title,449772,"10 The job title supplied by the Borrower when applying for the loan.* Name: Description, dtype: object","[nan, Yale University, Brick Township board of education, Classic Components, Compensation Solutions, Stanford University Libraries, LOCKSS Project, Macy's, Diamond Management and Technology Consultants, U.S. Bank, NC, College Pro Painters, Mana Products, Apto Solutions, Infinitely law group, Tanks Tavern, Air Force, Halping hands company inc., Homemaker, Evergreen Center, Town of Plainville, GA-PCOM, Tzigane Inc, Scheduall, Clinton Shop Rite of Hunterdon County, SUNY- ESF, 0CEAN VIEW INTL REALTY/C21 UNITED PLATINUM, Coldwell Banker Gordon Co. Realtors, Bozzuto Group, Walgreen's, Rock, Paper, Scissors, Best Buy, Eklektik Art Inc, Ernst & Young, North Carolina State University, McCamey Family, Outstanding Pool & Spa, Portrait Innovations, Subway, rombra us, inc, Gaston Memorial Hospital, jenny craig weight loss, Boston Architectural College, Department of Veterans Affairs, VUTEC, CORP, hf palm corp, MCHCP, Signs by Tomorrow, Squarewave Solutions, Ltd., FiSite Research, Target, Rush ..."
10,title,63154,"21 The loan title provided by the borrower Name: Description, dtype: object","[Buying a car, Debt consolidation, VISA, College Debt Consolidation, Credit Card, Starting a new job in a new city, Paying down high interest credit cards, Indonesia Underwater Photography, vacation loan, Moving Expenses for relocation, Credit card refinancing, Lowering My Interest Costs, Credit Card Payments, Credit card debt, Dep4774, Home improvement, Summer stuff, One-Debt Loan, Wedding coming up, delight, Car repair bill, Aroundthehouse, Consolidation Loan, roof, paying for medical school help, JAL Loan, MBA Loan Consolidation, Miscellanious, Consolidate credit cards, I need to pay off my parents for my car., CONSOLIDATING MY DEBTS, Moving expenses and security deposit, Credit Card Payoff, Get ahead on payments, New Bathroom, Pay costs of fixing car to attend school, Personal Loan, Consolidate Credit Card Debt, College Student Help, Young Professional Couple Finally Moving, Starting PhD, Consolidating Credit Cards, Pay 2 credit cards and Home Improvement, Consolidate 2 high-i..."
11,zip_code,953,"22 The first 3 numbers of the zip code provided by the borrower in the loan application. Name: Description, dtype: object","[100xx, 065xx, 087xx, 787xx, 078xx, 652xx, 080xx, 701xx, 452xx, 070xx, 806xx, 303xx, 021xx, 665xx, 024xx, 325xx, 064xx, 068xx, 027xx, 061xx, 300xx, 208xx, 017xx, 333xx, 088xx, 132xx, 651xx, 211xx, 019xx, 329xx, 014xx, 112xx, 276xx, 287xx, 462xx, 324xx, 331xx, 280xx, 115xx, 479xx, 820xx, 274xx, 802xx, 537xx, 020xx, 547xx, 041xx, 330xx, 339xx, 113xx, 805xx, 222xx, 306xx, 530xx, 469xx, 804xx, 314xx, 461xx, 950xx, 023xx, 069xx, 121xx, 207xx, 015xx, 282xx, 127xx, 334xx, 981xx, 321xx, 535xx, 245xx, 101xx, 531

- As you can see, there are some other datatypes like datetime, integer and boolean are mixed in the categorical column so we will fix it first
- Also, there are some irrelevant data like id or url, then, we will remove it

In [ ]:
# check for phase of completing each feature
# application = 1, funding = 2, approval = 3, finalization/labeling = 4
drop_features = ['pymnt_plan', 'hardship_flag', 'debt_settlement_flag', 'last_pymnt_d', 'last_credit_pull_d', 'grade', 'sub_grade']
cat_cols = [x for x in cat_col if x not in drop_features]
df = df.drop(drop_features, axis = 1)

In [ ]:
## remove two unrelated features id, url
df = df.drop('url', axis=1)
cat_cols.remove('url')

In [ ]:
df = df.drop('earliest_cr_line', axis=1)
## convert the numeric features that are mixed in categorical features for reducing cardinality
df['term'] = df['term'].map({" 36 months": 36, " 60 months": 60})
df['application_type'] = df['application_type'].map({"Individual": 1, "Joint App": 0})
df['application_type'].rename("is_individual_app", inplace=True)
df['initial_list_status'] = df['initial_list_status'].map({'w': 1, 'f': 0})
df['initial_list_status'].rename('is_whole_initial_list_status')
df['disbursement_method'] = df['disbursement_method'].map({'Cash': 1, 'DirectPay': 0})
df['disbursement_method'].rename("is_cash_disbursement_mt")
df['zip_code'] = df['zip_code'].apply(lambda x: int(str(x)[:3]) if pd.notnull(x) else np.nan)
cols_2_card = cat_dictionary[cat_dictionary['Cardinality'] == 2].Name.tolist() + ['zip_code' , 'earliest_cr_line']
cat_cols = [x for x in cat_cols if x not in cols_2_card]
num_col += ['term', 'is_individual_app', 'is_whole_initial_list_status', 'is_cash_disbursement_mt', 'zip_code']

In [ ]:
df.rename(columns={'application_type': 'is_individual_app',
                   'initial_list_status': 'is_whole_initial_list_status',
                   'disbursement_method': 'is_cash_disbursement_mt'}, inplace=True)

# PSI

In [ ]:
def calculate_psi(base, compare, bins=10, cat_threshold=10):
    """
    Calculate PSI for a single feature (continuous or categorical).
    base: Series (baseline, e.g. train)
    compare: Series (comparison, e.g. test)
    bins: number of quantile bins for continuous features
    cat_threshold: max unique values to treat as categorical
    """
    base = pd.Series(base)
    compare = pd.Series(compare)

    # --------------------------
    # Case 1: categorical / boolean
    # --------------------------
    if base.nunique(dropna=True) <= cat_threshold:
        categories = set(base.dropna().unique()) | set(compare.dropna().unique())
        base_counts = base.value_counts().reindex(categories, fill_value=0)
        compare_counts = compare.value_counts().reindex(categories, fill_value=0)
        # Add missing as a category
        base_counts["__missing__"] = base.isna().sum()
        compare_counts["__missing__"] = compare.isna().sum()

    # --------------------------
    # Case 2: continuous → quantile binning
    # --------------------------
    else:
        quantiles = np.linspace(0, 1, bins+1)
        cut_points = np.unique(np.quantile(base.dropna(), quantiles))
        if len(cut_points) < 2:
            return 0.0  # constant feature
        base_counts, _ = np.histogram(base.dropna(), bins=cut_points)
        compare_counts, _ = np.histogram(compare.dropna(), bins=cut_points)
        # Add missing as a bucket
        base_counts = np.append(base_counts, base.isna().sum())
        compare_counts = np.append(compare_counts, compare.isna().sum())

    # Convert to proportions
    base_perc = base_counts / (base_counts.sum() + 1e-6)
    compare_perc = compare_counts / (compare_counts.sum() + 1e-6)

    # PSI calculation
    psi = np.sum((compare_perc - base_perc) * np.log((compare_perc + 1e-6) / (base_perc + 1e-6)))
    return psi


def calculate_dataset_psi(train_df, test_df, features=None, bins=10, cat_threshold=10):
    """
    Calculate PSI for all features in a dataset.
    """
    if features is None:
        features = train_df.columns

    psi_results = {}
    for f in features:
        try:
            psi_results[f] = calculate_psi(train_df[f], test_df[f],
                                           bins=bins, cat_threshold=cat_threshold)
        except Exception as e:
            psi_results[f] = np.nan  # handle errors gracefully

    psi_df = pd.DataFrame(list(psi_results.items()), columns=["feature", "psi"])

    # Dataset-level summary
    summary = {
        "mean_psi": psi_df["psi"].mean(skipna=True),
        "median_psi": psi_df["psi"].median(skipna=True),
        "pct_over_0.1": (psi_df["psi"] > 0.1).mean(),
        "pct_over_0.25": (psi_df["psi"] > 0.25).mean()
    }

    return psi_df, summary
def frequency_encode(train, test, col):
    freqs = train[col].value_counts(normalize=True)
    train_encoded = train[col].map(freqs).fillna(0)
    test_encoded = test[col].map(freqs).fillna(0)  # unseen categories → 0
    return train_encoded, test_encoded

In [ ]:
cut_off = list(set([x for x in df.index.tolist() if x[-2:] == '01'])) + ['2007-01']
cut_off.sort()

In [ ]:
psi_over_years = {'features': num_col}
reversed_cut_off = cut_off[::-1][:-1]
for index, test_year in enumerate(reversed_cut_off):
  if index == 0: test_idx = df.index >= test_year
  else: test_idx = (df.index >= test_year) & (df.index < reversed_cut_off[index - 1])
  print(df.index[test_idx])
  print("----------------")
  for year in cut_off[:-(index + 1)]:
    print('Year train: ', year)
    train_idx = (df.index >= year) & (df.index < test_year)

    psi_df, _ = calculate_dataset_psi(df[train_idx], df[test_idx], num_col)
    psi_over_years[f'{year[:4]}-{test_year[:4]}'] = psi_df['psi'].apply(lambda x: f'{x:.2f}').astype('float64').tolist()


Index(['2017-01', '2017-01', '2017-01', '2017-01', '2017-01', '2017-01',
       '2017-01', '2017-01', '2017-01', '2017-01',
       ...
       '2017-12', '2017-12', '2017-12', '2017-12', '2017-12', '2017-12',
       '2017-12', '2017-12', '2017-12', '2017-12'],
      dtype='object', name='issue_d', length=443579)
----------------
Year train:  2007-01
Year train:  2008-01
Year train:  2009-01
Year train:  2010-01
Year train:  2011-01
Year train:  2012-01
Year train:  2013-01
Year train:  2014-01
Year train:  2015-01
Year train:  2016-01
Index(['2016-01', '2016-01', '2016-01', '2016-01', '2016-01', '2016-01',
       '2016-01', '2016-01', '2016-01', '2016-01',
       ...
       '2016-12', '2016-12', '2016-12', '2016-12', '2016-12', '2016-12',
       '2016-12', '2016-12', '2016-12', '2016-12'],
      dtype='object', name='issue_d', length=434407)
----------------
Year train:  2007-01
Year train:  2008-01
Year train:  2009-01
Year train:  2010-01
Year train:  2011-01
Year train:  2012-01
Year

- all tables show that data from 2013 is more stable let explore the reason

In [ ]:
psi_over_years = pd.DataFrame(psi_over_years).sort_values(by='2007-2017', ascending=False).reset_index().drop('index', axis=1)

In [ ]:
row_idx = 0
cols_to_highlight = ['2013-2014', '2013-2015', '2013-2016', '2013-2017']

def highlight_row_cols(x):
    df_style = pd.DataFrame('', index=x.index, columns=x.columns)
    for col in cols_to_highlight:
        if col in df_style.columns and row_idx in df_style.index:
            df_style.loc[row_idx, col] = 'background-color: yellow; color: black; font-weight: bold;'
    return df_style

# slice first, then apply styledisplay(
display(
    psi_over_years.loc[:, '2007-2017':'2016-2017'].head(1).style.apply(highlight_row_cols, axis=None)
)
display(
    psi_over_years.loc[:, '2007-2016':'2015-2016'].head(1).style.apply(highlight_row_cols, axis=None)
)
display(
    psi_over_years.loc[:, '2007-2015':'2014-2015'].head(1).style.apply(highlight_row_cols, axis=None)
)
display(
    psi_over_years.loc[:, '2007-2014':'2013-2014'].head(1).style.apply(highlight_row_cols, axis=None)
)


,2007-2017,2008-2017,2009-2017,2010-2017,2011-2017,2012-2017,2013-2017,2014-2017,2015-2017,2016-2017
0,0.630000,0.620000,0.600000,0.550000,0.450000,0.260000,0.040000,0.030000,0.020000,0.010000


,2007-2016,2008-2016,2009-2016,2010-2016,2011-2016,2012-2016,2013-2016,2014-2016,2015-2016
0,0.930000,0.930000,0.900000,0.830000,0.660000,0.380000,0.030000,0.020000,0.010000


,2007-2015,2008-2015,2009-2015,2010-2015,2011-2015,2012-2015,2013-2015,2014-2015
0,1.840000,1.830000,1.770000,1.640000,1.330000,0.750000,0.020000,0.010000


,2007-2014,2008-2014,2009-2014,2010-2014,2011-2014,2012-2014,2013-2014
0,3.970000,3.940000,3.840000,3.600000,2.990000,1.790000,0.010000


In [ ]:
df_2012 = df.loc[:'2012-12'].copy()
df_2013 = df.loc['2013-01':].copy()
x = len(df_2013[df_2013['num_rev_accts'].isna()]) / len(df_2013)
print("percentage of null values in column 'num_rev_accts' before 2013: ", len(df_2012[df_2012['num_rev_accts'].isna()]) / len(df_2012))
print("percentage of null values in column 'num_rev_accts after 2013: ", f'{x:.7f}')

percentage of null values in column 'num_rev_accts' before 2013:  0.7327897228420679
percentage of null values in column 'num_rev_accts after 2013:  0.0000006


In [ ]:
psi_cols = psi_over_years.columns.tolist()[1:]
col_2013 = [x for x in psi_cols if int(x[:4]) >= 2013]

In [ ]:
np.sum(np.round(psi_over_years[col_2013], 1) > 0.5).sum()

/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


np.int64(0)

In [ ]:
df = df[df.index >= '2013-01'].copy()

In [ ]:
train = df[df.index < '2017-01']
test = df[df.index >= '2017-01']

In [ ]:
train[cat_cols]

,emp_title,emp_length,home_ownership,verification_status,purpose,title,addr_state
issue_d,,,,,,,
2013-01,donaldson company,10+ years,MORTGAGE,Not Verified,vacation,Vacation,WI
2013-01,MD STAT URGENT CARE,3 years,RENT,Source Verified,debt_consolidation,Debt Consolidation/ Wedding,CA
2013-01,Evolve Capital Ltd,6 years,MORTGAGE,Verified,debt_consolidation,Debt Consolidation,TX
2013-01,NaN,NaN,MORTGAGE,Verified,other,Catch-Up,CO
2013-01,Microsoft,7 years,MORTGAGE,Verified,debt_consolidation,2013Consolidate,WA
...,...,...,...,...,...,...,...
2016-12,General Manager,3 years,ANY,Not Verified,debt_consolidation,Debt consolidation,VA
2016-12,NaN,NaN,MORTGAGE,Source Verified,house,Home buying,NY
2016-12,Operations Manager,6 years,RENT,Source Verified,debt_consolidation,Debt consolidation,TX


In [ ]:
freq_encode_cols = ['emp_title', 'title', 'addr_state']
for col in freq_encode_cols:
  train[col], test[col] = frequency_encode(train, test, col)

/tmp/ipython-input-1829850974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col], test[col] = frequency_encode(train, test, col)
/tmp/ipython-input-1829850974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[col], test[col] = frequency_encode(train, test, col)
/tmp/ipython-input-1829850974.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [ ]:
calculate_dataset_psi(train, test, cat_cols, cat_threshold=15)

(               feature       psi
 0            emp_title  0.470183
 1           emp_length  0.021192
 2       home_ownership  0.002813
 3  verification_status  0.029381
 4              purpose  0.026898
 5                title  0.523672
 6           addr_state  0.000409,
 {'mean_psi': np.float64(0.15350674594406696),
  'median_psi': 0.026897586058746998,
  'pct_over_0.1': np.float64(0.2857142857142857),
  'pct_over_0.25': np.float64(0.2857142857142857)})

- so from 2013 data features are more stable (psi <= 0.5) so I will choose data from this time to train

- psi of emp_title, title is high can be the result of high cardinality

In [ ]:
import re

def clean_title(text):
    if pd.isnull(text):
        return "unknown"
    text = text.lower().strip()
    text = re.sub(r'[^a-z ]', '', text)   # keep only letters and space
    text = re.sub(r'\bsr\b|\bsenior\b', 'senior', text)  # unify "sr"/"senior"
    text = re.sub(r'\bjr\b|\bjunior\b', 'junior', text)  # unify "jr"/"junior"
    return text



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
high_card_train = df.loc[df.index < '2017-01', ['emp_title', 'title', 'addr_state']]
high_card_test = df.loc[df.index >= '2017-01', ['emp_title', 'title', 'addr_state']]

for col in high_card_train.columns:
  # Vectorize job titles
  vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")
  high_card_train[col].apply(clean_title)
  high_card_test[col].apply(clean_title)
  X_train = vectorizer.fit_transform(high_card_train[col].fillna(""))
  X_test = vectorizer.transform(high_card_test[col].fillna(""))
  # Cluster into, say, 10 groups
  kmeans = KMeans(n_clusters=10, random_state=42)
  high_card_train[col] = kmeans.fit_predict(X_train)
  high_card_test[col] = kmeans.predict(X_test)

In [ ]:
calculate_dataset_psi(high_card_train, high_card_test, ['emp_title', 'title', 'addr_state', 'sub_grade'])

(      feature       psi
 0   emp_title  0.001948
 1       title  0.114256
 2  addr_state  0.000385
 3   sub_grade       NaN,
 {'mean_psi': np.float64(0.03886308035911633),
  'median_psi': 0.001948178362841589,
  'pct_over_0.1': np.float64(0.25),
  'pct_over_0.25': np.float64(0.0)})

the main reason leading to high psi is high cardinality -> we should group the high cardinality categorical features in data engineering

In [ ]:
train_df = df[df.index < '2017-01']
test_df = df[df.index >= '2017-01']
recoveries_train = recoveries[(recoveries.index >= '2013-01') & (recoveries.index < '2017-01')]
recoveries_test = recoveries[recoveries.index >= '2017-01']
train_df.to_csv('/content/drive/MyDrive/project/data/train_data.csv')
test_df.to_csv('/content/drive/MyDrive/project/data/test_data.csv')
recoveries_train.to_csv('/content/drive/MyDrive/project/data/train_recoveries.csv')
recoveries_test.to_csv('/content/drive/MyDrive/project/data/test_recoveries.csv')